# Random forest as an ensemble of decision trees

_Kevin Siswandi_  
**Fundamentals of Machine Learning**  
December 2019  

From mathematical point of view, random forest is not well calibrated: it is usually overconfident and the posterior probability estimate resembles more of a step function that overshoots the true posterior probability. For this reason, we should prefer neural network when training data is abundant. However, random forest has been proven to work really well when the training data are limited (as in our case) or computational resources are constrained.

In particular, random forest gives high prediction accuracy without parameter tuning, fast at train and test time, and has an in-built accuracy measure and interpretation tool that we can use to explain the predictions.

## Training complexity

The computational complexity of tree construction for *monothetic* decision trees is as follows.
- At the root node (level 0)
    * sort $n$ samples according to each of $p$ features.
    * measure impurity for each split position according to each of $p$ features.

Since sorting is $\mathcal{O}(n \log{n})$ and measurement of impurity is $\mathcal{O}(n)$, we have a total complexity of $\mathcal{O}(p n \log{n})$ at the root node. Now, if each child receives half of the samples, then at level 1, we have for the left child and right child:
$$ \begin{split} \mathcal{O}\left(p\frac{n}{2} \log{\frac{n}{2}} + p\frac{n}{2} \log{\frac{n}{2}} \right) \\
 = \mathcal{O}\left(p n \log{\frac{n}{2}} \right)
\end{split}$$

By recursive argument, we have $\mathcal{O}\left(p n \log{\frac{n}{4}} \right)$ for level 2, and so on. Therefore, for a balanced binary tree with $\lg n$ levels (logarithm with base 2), overall training complexity is $ \mathcal{O}\left(p n (\lg{n})^2 \right) $. During prediction, the computational complexity is:
+ best case: $\lg n$, when the tree always sends half the samples to the left child and the other half to the right child. For example, 1024 samples would need a tree of depth 10.
+ worst case: $n$, when each split sends only one sample to one of the children every time.
+ average case: $\mathcal{O}(\lg n)$, i.e. the tree is usually not very imbalanced. Note that this is cheap: c.f. k-NN with cost $\mathcal{O}(n)$.

Note: balanced tree = each leaf is at the same level of the tree.

## Axis-orthogonal split

The most popular kind of splits is axis-orthogonal, where only a single feature is considered at every node/split. The advantage of an axis-orthogonal split: it is invariant w.r.t. scaling of the axes, i.e. choice of feature units can be arbitrary. However, the drawback is that several splits are required for correlated features. This can be overcome by introducing *oblique* splits, which ask if the inner product of some weight vector and the feature vector satisfies a condition:

$$ w^T x > b $$

Alternatively, one could preprocess the data with Principal Component Analysis (PCA). Nevertheless, a decision tree is likely to overfit if it is trained to purity because it would do several splits to account for each observation. Thus it has low bias but high variance. Two common regularization methods to address the high variance problem are:
1. *Early stopping*: abort if node has too few samples, or if impurity reduction is too small, etc.
2. *Pruning*: build a tree and cut it back.

Nevertheless, the state-of-the-art method to reduce variance of decision trees while maintaining the low bias is to create an ensemble of decision trees (a.k.a. **random forest**).

## Random Forest

Random Forest is proposed by [Leo Breiman, 2001](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm) and works by injecting two sources of randomness:
1. sample with replacement from original training data (i.e. bootstrapping) such that each tree gets different training data. In ML speak, this is called *bagging* (quasi-short for bootstrap aggregation).
2. consider only a random subset of features to find the "optimal" split in each tree, at each node.

Due to the bagging procedure, there are samples that a tree has not seen during training time. One can use these unseen samples to make predictions to get **out-of-bag** error, which gives an in-built estimate of generalization performance (c.f. cross-validation). Now due to random split selection, one can count, for each feature, how often it is used for splitting: this gives a measure of how important a feature is in making predictions (**Gini importance**). Note that Gini importance is biased towards features with many categorical levels, but it is still useful and cheap to compute.

Prediction can be made using majority vote. However, an artifact of this approach is that the estimates in the "hard" regions where impurity is relatively high cannot be interpreted as posterior probability, because random forest is empirically overconfident.

Additional remarks:
- In greedy tree construction, one iteratively finds the (local) best split at each iteration.
- Bagging and random split selection are the two key factors that enable random forest.
- Two hyperparameters of random forest: number of trees and size of random subset. However usually it works fine with the default values (256 trees and $\lg p$ feature subset) and tuning these values do not have significant impact on performance.
- RF can handle both categorical and continuous features simultaneuously